In [1]:
import os
os.chdir(r'7 - Putting it together')
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import random
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.tensorboard import SummaryWriter
from itertools import chain, cycle, groupby
from functools import reduce
from typing import Collection, List
from pathlib import Path
import music21 as m21
musescore_path = '/usr/bin/mscore'
m21.environment.set('musicxmlPath', musescore_path)
m21.environment.set('musescoreDirectPNGPath', musescore_path)
from midi_encoding import *
from data_loading import *
from model import *
from einops import rearrange, repeat, pack, unpack, einsum
import faiss
import time
import math
import pickle

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device}.")

Using cuda.


In [2]:
!nvidia-smi

Tue Sep 10 18:28:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
| 30%   33C    P2             49W /  450W |    3309MiB /  24564MiB |     43%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
vg_large_path = Path('../data/midi/vg_large')
vg_large_file_names = [f for f in os.listdir(vg_large_path) if os.path.isfile(os.path.join(vg_large_path, f))]

# Ensure files are shuffled directly after assignment.
# If they are shuffled in a different cell, and that cell is run multiple times, the order will change as we are shuffling the already-shuffled list.
random.seed(42)
random.shuffle(vg_large_file_names)

len(vg_large_file_names)

3839

In [4]:
sample_length = 256
max_file_length = 32

midi_path = Path('../data/midi/vg_large')
score_path = Path(f'../data/numpy/vg_large/all')

n1 = int(0.8 * len(vg_large_file_names))
n2 = int(0.9 * len(vg_large_file_names))
train_filenames = vg_large_file_names[:n1]
valid_filenames = vg_large_file_names[n1:n2]
test_filenames = vg_large_file_names[n2:]

print(f'Train file names: {len(train_filenames)}, Valid file names: {len(valid_filenames)}, Test file names: {len(test_filenames)}')

Train file names: 3071, Valid file names: 384, Test file names: 384
